In [0]:
import pandas as pd
from pprint import pprint

In [44]:
df_indicadores = pd.read_csv('dados_oceans14_liquidos.csv')
df_indicadores.columns

Index(['tick', 'ano', 'divida_liq_EBITDA', 'ROIC', 'VPA', 'ROA', 'P_L', 'PSR',
       'P_VP', 'ECP', 'P_EBIT', 'DY', 'ROE', 'P_ativos', 'margem_EBITDA',
       'LPA', 'margem_liq', 'EV_EBITDA', 'beta', 'P_EBITDA', 'EV_EBIT'],
      dtype='object')

In [0]:
def somar_posicoes(df_rank, list_indicadores):
    linhas_posi = []
    for tick, linha in df_rank.iterrows():
        soma_posi = 0
        for i in range(1, len(linha), 1):
            soma_posi += linha[i]
        linhas_posi.append([tick, soma_posi])
    
    df_posi = pd.DataFrame(linhas_posi, columns=['tick', 'soma_posi'])
    df_posi.set_index('tick', inplace=True)
    return df_rank.join(df_posi).copy()

In [0]:
def calcular_ranking(df_completo, list_indicadores, ano):
    '''
    list_indicadores = [['TICKER', bool (O primeiro lugar é o menor valor?)], [Ticker, bool], ...]
    '''
    df_rankings = df_completo[df_completo['ano'] == ano][['tick', 'ano']].copy()
    df_rankings.set_index('tick', inplace=True)

    for indicador, asc in list_indicadores:
        df_indicador = df_completo[df_completo['ano'] == ano][['tick', indicador]].copy()
        df_indicador.set_index('tick', inplace=True)

        df_indicador['rank_'+indicador] = df_indicador[indicador].rank(ascending=asc)
        df_indicador.drop(indicador, axis=1, inplace=True)

        df_rankings = df_rankings.join(df_indicador)
    
    df_rankings = somar_posicoes(df_rankings, list_indicadores)
    return df_rankings.sort_values('soma_posi').copy()

In [0]:
def analisar_indicadores(lista_indicadores, ano):
    DICT_ASC = {
        'divida_liq_EBITDA': ['divida_liq_EBITDA', True],
        'ROIC': ['ROIC', False],
        'VPA': ['VPA', True],
        'ROA': ['ROA', False],
        'P_L': ['P_L', True],
        'PSR': ['PSR', False],
        'P_VP': ['P_VP', True],
        'P_EBIT': ['P_EBIT', True],
        'DY': ['DY', False],
        'ROE': ['ROE', False],
        'P_ativos': ['P_ativos', True],
        'margem_EBITDA': ['margem_EBITDA', False],
        'LPA': ['LPA', False],
        'margem_liq': ['margem_liq', False],
        'EV_EBITDA': ['EV_EBITDA', True],
        'P_EBITDA': ['P_EBITDA', True],
        'EV_EBIT': ['EV_EBIT', True],
    }

    lista_duplas = []
    for indicador in lista_indicadores:
        lista_duplas.append(DICT_ASC[indicador])
    
    df_indicadores = pd.read_csv('dados_oceans14_liquidos.csv')
    
    df_final = calcular_ranking(df_indicadores, lista_duplas, ano)

    df_final['posi_normalizada'] = round((100*df_final['soma_posi'])/(len(df_final)*len(indicadores)),2)

    colunas_certas = list(df_final.columns)
    colunas_certas.remove('ano')
    colunas_certas.remove('soma_posi')
    colunas_certas.remove('posi_normalizada')
    colunas_certas = ['soma_posi', 'posi_normalizada'] + colunas_certas

    return df_final[colunas_certas].copy()


In [48]:
indicadores = ['ROIC', 'EV_EBITDA']
ano = 2019
df_final = analisar_indicadores(indicadores, ano)
df_final.head(10)

,soma_posi,posi_normalizada,rank_ROIC,rank_EV_EBITDA
tick,,,,
BBSE3,14.0,4.24,1.0,13.0
WIZS3,15.0,4.55,2.0,13.0
SMLS3,19.0,5.76,3.0,16.0
CEAB3,28.5,8.64,8.5,20.0
GOLL4,33.5,10.15,14.5,19.0
CNTO3,38.5,11.67,25.5,13.0
PRIO3,42.5,12.88,12.5,30.0
TGMA3,54.5,16.52,6.0,48.5
CGRA4,58.5,17.73,30.5,28.0
